In [46]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder 
from sklearn.svm import SVC

In [33]:
#Cargo los archivos
eventos = pd.read_csv("../events_up_to_01062018.csv")
labels = pd.read_csv("../labels_training_set.csv")

C:\Users\felip\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
#Veo las columnas de eventos 
list(eventos)

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [65]:
#Dropeo columnas que son irrelevantes
train_set = eventos.drop(columns=['city','staticpage','url','timestamp','color','skus', 'channel','search_term','campaign_source','search_engine','screen_resolution','operating_system_version','browser_version'])
train_set.head()

,event,person,sku,model,condition,storage,new_vs_returning,region,country,device_type
0,viewed product,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,NaN,NaN,NaN,NaN
1,viewed product,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,NaN,NaN,NaN,NaN
2,viewed product,0297fc1e,6888.0,iPhone 6S,Muito Bom,64GB,NaN,NaN,NaN,NaN
3,viewed product,2d681dd8,11890.0,iPhone 7,Bom,128GB,NaN,NaN,NaN,NaN
4,viewed product,cccea85e,7517.0,LG G4 H818P,Excelente,32GB,NaN,NaN,NaN,NaN


In [66]:
#Me quedo con los set de datos que coincidan con los labels y sean inferiores al 01/06/2018
set_entrenamiento = pd.merge(train_set, labels, on='person', how='inner')


In [67]:
list(set_entrenamiento)

['event',
 'person',
 'sku',
 'model',
 'condition',
 'storage',
 'new_vs_returning',
 'region',
 'country',
 'device_type',
 'label']

In [72]:
#Encodeamos columnas Strings

lb_make = LabelEncoder()
set_entrenamiento['event'] = lb_make.fit_transform(set_entrenamiento['event'].astype(str))
set_entrenamiento['sku'] = lb_make.fit_transform(set_entrenamiento['sku'].astype(str))
set_entrenamiento['model'] = lb_make.fit_transform(set_entrenamiento['model'].astype(str))

set_entrenamiento['person'] = lb_make.fit_transform(set_entrenamiento['person'].astype(str))
set_entrenamiento['condition'] = lb_make.fit_transform(set_entrenamiento['condition'].astype(str))
set_entrenamiento['storage'] = lb_make.fit_transform(set_entrenamiento['storage'].astype(str))
set_entrenamiento['new_vs_returning'] = lb_make.fit_transform(set_entrenamiento['new_vs_returning'].astype(str))
set_entrenamiento['region'] = lb_make.fit_transform(set_entrenamiento['region'].astype(str))
set_entrenamiento['country'] = lb_make.fit_transform(set_entrenamiento['country'].astype(str))
set_entrenamiento['device_type'] = lb_make.fit_transform(set_entrenamiento['device_type'].astype(str))

set_entrenamiento.head()


,event,person,sku,model,condition,storage,new_vs_returning,region,country,device_type,label
0,10,13068,119,98,3,3,2,83,33,4,0
1,10,13068,157,98,3,6,2,83,33,4,0
2,10,13068,549,98,1,1,2,83,33,4,0
3,10,13068,559,98,1,1,2,83,33,4,0
4,0,13068,1359,110,5,8,2,83,33,4,0


In [73]:
##Empezamos a cronstruir el modelo. Nos basamos en un ejemplo

# Split-out validation dataset
array = set_entrenamiento.values
X = array[:,0:10]
Y = array[:,10]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [74]:
scoring = 'accuracy'

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

LR: 0.914817 (0.000607)
LDA: 0.914817 (0.000607)
